In [1]:
import json

In [2]:
def json_load_file(file):
    """
    Returns a Python's dict from a json file path.
    """
    with open(file) as json_file:
        return json.load(json_file)

In [3]:
def make_students_ids_map(object_json):
    """
    Creates a mapping from student id (number) to its hash.
    
    For example:
    1 -> TTA3wnBvnrgbegTGL
    2 -> YR3xW7MSKf3Cn32g8
    ...
    """
    result = {}
    for student_hash, student_name in object_json["globalStructure"]["students"].items():
        try:
            student_id = int(student_name)
            result[student_id] = student_hash
        except ValueError:
            pass
    return result

In [4]:
def get_payload(product_json):
    """
    Gets either product_json["payload"] or product_json["activityData"]["payload"] (!)
    """
    if "activityData" in product_json:
        return product_json["activityData"]["payload"]
    return product_json["payload"]

In [5]:
def extract_answers(questions_json):
    """
    Parses answers from a questions JSON ({"answersIndex": number[], "form": ...}) to an well-formed array.
    """
    result = []
    for i, answer_index in enumerate(questions_json["answersIndex"]):
        if answer_index == -1:
            if str(i) in questions_json["form"]:
                result.append(questions_json["form"][str(i)]["text"])
            else:
                result.append(None)
        else:
            result.append(answer_index)
    return result

In [6]:
def make_students_answers_map(students_ids_map, product_json):
    """
    Creates a mapping from student id (number) to its answers (array).
    
    For example:
    1 -> ["<?-+>?", "[0-9]*[0,2,4,6,8]", ...]
    ...
    """
    result = {}
    for student_id, student_hash in students_ids_map.items():
        answers = extract_answers(get_payload(product_json)[student_hash]["data"])
        if answers[0]: # only if answers is not empty
            result[student_id] = answers
    return result

In [7]:
for school in ["piccard", "beaulieu"]:
    data_path = f"../data/raw/{school}/ac/5-final-quizz/"
    students_ids_map = make_students_ids_map(json_load_file(f"{data_path}object.json"))
    students_answers = make_students_answers_map(students_ids_map, json_load_file(f"{data_path}product.json"))
    output_path = f"../data/derived/{school}/final_quizz.json"
    with open(output_path, 'w+') as output_file:
        json.dump(students_answers, output_file, sort_keys=True, indent=4)
    print("# " + school + " #")
    for student_id, answers in sorted(students_answers.items()):
        print(student_id)
        for answer in answers:
            print("\t" + str(answer))

# piccard #
1
	(\<)?(\-)+(\>)?

J'ai bien aimé cette présentation car elle était ludique.
	[0-9]+[0|2|4|6|8]
	[0-9]+(\.[0-9]+)?
	Hey o
HeyHey o
HeyHey UhUh
HeyHey ooo
HeyHey Uh
	1
	2
	None
2
	(\<)?(\-)+(\>)?


Chouette cours, les notions sont explicitées de manière extrêmement compréhensible, le concours est cool, merci!!
	[1-9]*[2|4|6|8|0]
	[0-9]+(\.)?[0-9]+
	Hey, HeyHey O, HeyHeyUh, HeyHey Uh, HeyHeyHeyUh,
	1
	2
	None
3
	[<|>]?[-]+[<|>]?

Bon concept de pouvoir interagir avec les autres groupes, sujet intéressant, concours avec les boules Lindt très instructif ;) Merci!

	[2,4,6,8]?[0,2,4,6,8]+
	[0-9],[0-9]?
	Hey, Hey O, Hey Uh, Hey UhUh, Hey OhOh
	1
	2
	None
7
	<?-+>?
	[0-9]+(0|2|4|6|8)
	[0-9]+(.|,)[0-9]+
	Hey
Hey Hey
Hey Uh
Hey Uh Uh
Hey Hey O Uh
	0
	2
	None
8
	[-]+[<]?[>]?
	[0-9]+[0|2|4|6]
	[0-9]+.[0-9]*
	Hey,Hey , HeyO, Hey O, Hey Uh
	0
	2
	None
9
	[<|>|-][-]+[<|>]
	[0-9]+[0|2|4|6|8]

	[0-9]+[.][0-9]*
	Hey O, Hey Hey Hey, Hey Hey Uh
	1
	2
	None
10
	<?+?>+?-
	[0-9]*[2,4,6,8]
	[0-9